<a href="https://colab.research.google.com/github/Deszynski/PRiR-lab9/blob/main/PRiR-lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

%load_ext nvcc_plugin


In [47]:
%%cu
#include <stdio.h>
#include <stdlib.h>
 
#define MAX_THREADS 128 
#define N 512

int* r_values;
int* d_values;

void Init(int* values, int i) 
{
    srand( time(NULL) );
    printf("\n------------------------------\n");

    if (i == 0) 
    {
        printf("Data set distribution: Uniform\n");
        for (int x = 0; x < N; ++x) 
		{
            values[x] = rand() % 100;
            //printf("%d ", values[x]);
        }
    }
    else if (i == 1) 
    {
        #define MEAN 100
        #define STD_DEV 5
        printf("Data set distribution: Gaussian\n");
        float r;
        for (int x = 0; x < N; ++x) 
		{
            r  = (rand()%3 - 1) + (rand()%3 - 1) + (rand()%3 - 1);
            values[x] = int( round(r * STD_DEV + MEAN) );
            //printf("%d ", values[x]);
        }
    }
    else if (i == 2) 
    {
        printf("Data set distribution: Bucket\n");
        int j = 0;
        for (int x = 0; x < N; ++x, ++j) 
		{
            if (j / 20 < 1)
                values[x] = rand() % 20;
            else if (j / 20 < 2)
                values[x] = rand() % 20 + 20;
            else if (j / 20 < 3)
                values[x] = rand() % 20 + 40;
            else if (j / 20 < 4)
                values[x] = rand() % 20 + 60;
            else if (j / 20 < 5)
                values[x] = rand() % 20 + 80;
            if (j == 100)
                j = 0;
            //printf("%d ", values[x]);
        }
    }
    else if (i == 3) 
    {
        printf("Data set distribution: Sorted\n");
        for (int x = 0; x < N; ++x)
            printf("%d ", values[x]);
    }
	else if (i == 4) 
    {
        printf("Data set distribution: Zero\n");
        int r = rand() % 100;
        for (int x = 0; x < N; ++x) 
		{
            values[x] = r;
            //printf("%d ", values[x]);
        }
    }       
	printf("\n");
}

 // kernel
 __global__ static void quicksort(int* values) 
 {
 	#define MAX_LEVELS	300
	int pivot, L, R;
	int idx =  threadIdx.x + blockIdx.x * blockDim.x;
	int start[MAX_LEVELS];
	int end[MAX_LEVELS];

	start[idx] = idx;
	end[idx] = N - 1;
	while (idx >= 0) 
	{
		L = start[idx];
		R = end[idx];
		if (L < R) 
		{
			pivot = values[L];
			while (L < R) 
			{
				while (values[R] >= pivot && L < R)
					R--;
				if(L < R)
					values[L++] = values[R];
				while (values[L] < pivot && L < R)
					L++;
				if (L < R)
					values[R--] = values[L];
			}
			values[L] = pivot;
			start[idx + 1] = L + 1;
			end[idx + 1] = end[idx];
			end[idx++] = L;
			if (end[idx] - start[idx] > end[idx - 1] - start[idx - 1]) 
			{
	            // zamien start[idx] z start[idx-1]
        	    int tmp = start[idx];
                start[idx] = start[idx - 1];
                start[idx - 1] = tmp;

	            // zamien end[idx] z end[idx-1]
        	    tmp = end[idx];
                end[idx] = end[idx - 1];
                end[idx - 1] = tmp;
	        }
		}
		else
			idx--;
	}
}

int main(int argc, char **argv) 
{
	printf("Quicksort begins with %d nunmbers...\n", N);
 	unsigned int hTimer;
 	size_t size = N * sizeof(int);
 	
 	// allocate host memory
 	r_values = (int*)malloc(size);
 	
	// allocate device memory
    cudaMalloc((void**)&d_values, size);

	// allocate threads per block
    const unsigned int cThreadsPerBlock = 128;
                
	/* Types of data sets to be sorted:
        1. Normal distribution
        2. Gaussian distribution
        3. Bucket distribution
        4. Sorted Distribution
        5. Zero Distribution
    */

	for (int i = 0; i < 5; ++i) 
	{
        // initialize data set
        Init(r_values, i);

	 	// copy data to device	
		cudaMemcpy(d_values, r_values, size, cudaMemcpyHostToDevice);

		printf("Beginning kernel execution...\n");

		cudaEvent_t start, stop;
		cudaEventCreate( &start );
		cudaEventCreate( &stop );
		cudaEventRecord( start, 0 );
	
		// execute kernel
 		quicksort <<< MAX_THREADS / cThreadsPerBlock, MAX_THREADS / cThreadsPerBlock, cThreadsPerBlock >>> (d_values);

 		cudaThreadSynchronize();

		cudaEventRecord( stop, 0 );
		cudaEventSynchronize( stop );
		float elapsedTime;
		cudaEventElapsedTime( &elapsedTime, start, stop );
		printf( "Elapsed time: %3.1f ms\n", elapsedTime );
		cudaEventDestroy( start );
		cudaEventDestroy( stop );

 	
	 	// copy data back to host
		cudaMemcpy(r_values, d_values, size, cudaMemcpyDeviceToHost);
 	
	 	// test print
 		for (int i = 0; i < N; i++) 
		{
 			printf("%d ", r_values[i]);
 		}
 		
 		printf("\n");
		
		// test
    	printf("\nTesting results...\n");
    	for (int x = 0; x < N - 1; x++) 
    	{
        	if (r_values[x] > r_values[x + 1]) 
        	{
            	printf("Sorting failed.\n");
            	break;
        	}
        	else if (x == N - 2)
            	printf("SORTING SUCCESSFUL\n");
    	}
	}
		
 	// free memory
	cudaFree(d_values);
 	free(r_values);
 	cudaThreadExit();
}

Quicksort begins with 512 nunmbers...

------------------------------
Data set distribution: Uniform

Beginning kernel execution...
Elapsed time: 10.3 ms
0 0 0 0 0 1 1 2 2 2 3 3 3 3 3 3 4 4 4 5 5 5 5 5 5 5 5 5 6 6 6 6 6 7 7 7 7 8 8 8 8 8 8 8 8 8 9 9 9 10 10 10 10 10 10 11 11 11 11 11 12 12 12 12 12 13 13 13 13 13 14 14 14 14 14 15 15 15 15 15 15 16 16 16 16 16 16 16 16 16 16 16 17 17 17 17 17 17 18 18 18 18 19 19 19 19 20 20 20 20 20 21 21 21 21 21 21 21 21 21 22 22 22 22 22 22 23 23 23 23 24 24 24 24 24 24 25 25 25 25 25 25 25 25 25 26 26 26 26 26 26 26 27 28 28 28 28 28 28 28 28 29 29 29 29 29 29 30 30 31 32 32 32 32 33 33 33 33 33 34 34 34 34 35 35 35 35 35 35 36 36 36 36 37 37 37 37 37 37 38 38 38 38 38 39 39 39 40 41 41 41 41 41 41 41 42 42 42 42 42 43 43 43 43 43 43 44 44 44 44 44 44 44 45 45 45 45 45 46 46 46 47 47 47 47 47 47 48 48 48 48 49 49 49 49 49 49 49 50 50 50 50 50 50 50 51 51 51 51 51 52 52 52 52 52 53 53 53 53 53 53 54 54 54 54 54 54 55 55 55 55 55 56 56 56 56 56 57 5

In [46]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <chrono>
#include <sys/time.h>
 
#define MAX_THREADS 128 
#define N 512

 int* r_values;
 int* d_values;

void Init(int* values, int i) 
{
    srand( time(NULL) );
    printf("\n------------------------------\n");

    if (i == 0) 
    {
        printf("Data set distribution: Uniform\n");
        for (int x = 0; x < N; ++x) 
		{
            values[x] = rand() % 100;
            //printf("%d ", values[x]);
        }
    }
    else if (i == 1) 
    {
        #define MEAN 100
        #define STD_DEV 5
        printf("Data set distribution: Gaussian\n");
        float r;
        for (int x = 0; x < N; ++x) 
		{
            r  = (rand()%3 - 1) + (rand()%3 - 1) + (rand()%3 - 1);
            values[x] = int( round(r * STD_DEV + MEAN) );
            //printf("%d ", values[x]);
        }
    }
    else if (i == 2) 
    {
        printf("Data set distribution: Bucket\n");
        int j = 0;
        for (int x = 0; x < N; ++x, ++j) 
		{
            if (j / 20 < 1)
                values[x] = rand() % 20;
            else if (j / 20 < 2)
                values[x] = rand() % 20 + 20;
            else if (j / 20 < 3)
                values[x] = rand() % 20 + 40;
            else if (j / 20 < 4)
                values[x] = rand() % 20 + 60;
            else if (j / 20 < 5)
                values[x] = rand() % 20 + 80;
            if (j == 100)
                j = 0;
            //printf("%d ", values[x]);
        }
    }
    else if (i == 3) 
    {
        printf("Data set distribution: Sorted\n");
        for (int x = 0; x < N; ++x)
            printf("%d ", values[x]);
    }
	else if (i == 4) 
    {
        printf("Data set distribution: Zero\n");
        int r = rand() % 100;
        for (int x = 0; x < N; ++x) 
		{
            values[x] = r;
            //printf("%d ", values[x]);
        }
    }       
	printf("\n");
}

 // kernel
static void quicksort(int* values) 
{
 	#define MAX_LEVELS	300
	int pivot, L, R;
	int idx =  0;
	int start[MAX_LEVELS];
	int end[MAX_LEVELS];

	start[idx] = idx;
	end[idx] = N - 1;
	while (idx >= 0) 
	{
		L = start[idx];
		R = end[idx];
		if (L < R) 
		{
			pivot = values[L];
			while (L < R) 
			{
				while (values[R] >= pivot && L < R)
					R--;
				if(L < R)
					values[L++] = values[R];
				while (values[L] < pivot && L < R)
					L++;
				if (L < R)
					values[R--] = values[L];
			}
			values[L] = pivot;
			start[idx + 1] = L + 1;
			end[idx + 1] = end[idx];
			end[idx++] = L;
			if (end[idx] - start[idx] > end[idx - 1] - start[idx - 1]) 
			{
	            // zamien start[idx] z start[idx-1]
        	    int tmp = start[idx];
                start[idx] = start[idx - 1];
                start[idx - 1] = tmp;

	            // zamien end[idx] z end[idx-1]
        	    tmp = end[idx];
                end[idx] = end[idx - 1];
                end[idx - 1] = tmp;
	        }
		}
		else
			idx--;
	}
}

int main(int argc, char **argv) 
{
	printf("Quicksort begins with %d nunmbers...\n", N);
 	unsigned int hTimer;
 	size_t size = N * sizeof(int);
 	
 	// allocate host memory
 	r_values = (int*)malloc(size);

	// allocate threads per block
    const unsigned int cThreadsPerBlock = 128;
                
	/* Types of data sets to be sorted:
        1. Normal distribution
        2. Gaussian distribution
        3. Bucket distribution
        4. Sorted Distribution
        5. Zero Distribution
    */

	for (int i = 0; i < 5; ++i) 
	{
        // initialize data set
        Init(r_values, i);

		printf("Beginning kernel execution...\n");
			
		cudaEvent_t start, stop;
		cudaEventCreate( &start );
		cudaEventCreate( &stop );
		cudaEventRecord( start, 0 );

		// execute kernel
 		quicksort(r_values);
		
		cudaEventRecord( stop, 0 );
		cudaEventSynchronize( stop );
		float elapsedTime;
		cudaEventElapsedTime( &elapsedTime, start, stop );
		printf( "Elapsed time: %3.1f ms\n", elapsedTime );
		cudaEventDestroy( start );
		cudaEventDestroy( stop );
 	
	 	// test print
 		for (int i = 0; i < N; i++) 
		{
 			printf("%d ", r_values[i]);
 		}
 		
 		printf("\n");
		
		// test
    	printf("\nTesting results...\n");
    	for (int x = 0; x < N - 1; x++) 
    	{
        	if (r_values[x] > r_values[x + 1]) 
        	{
            	printf("Sorting failed.\n");
            	break;
        	}
        	else if (x == N - 2)
            	printf("SORTING SUCCESSFUL\n");
    	}
	}
}

Quicksort begins with 512 nunmbers...

------------------------------
Data set distribution: Uniform

Beginning kernel execution...
Elapsed time: 0.0 ms
0 0 0 0 0 0 0 1 1 1 2 2 2 2 2 3 3 3 3 3 4 4 4 4 5 5 5 5 5 5 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 8 8 8 9 9 9 9 10 10 10 10 10 10 11 11 11 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13 14 14 14 14 14 14 15 15 15 15 16 16 16 16 17 17 17 17 17 17 17 17 18 18 18 18 18 18 19 19 19 20 20 20 20 20 20 20 21 21 21 21 21 21 21 21 22 22 23 23 23 23 23 24 24 24 24 24 24 24 25 25 25 26 26 26 26 27 27 27 27 27 28 28 28 28 28 28 28 28 29 29 29 29 29 29 29 29 29 30 30 30 30 31 31 31 31 31 31 31 32 32 32 32 32 33 33 33 33 33 33 33 33 33 33 34 34 34 34 34 34 34 35 35 35 35 35 35 35 35 36 37 37 37 37 37 37 37 37 38 38 38 38 38 38 39 39 39 39 39 40 40 40 40 41 42 42 42 42 43 43 43 43 43 43 44 44 44 44 44 44 45 45 45 45 45 46 46 46 46 47 47 47 48 48 48 48 48 49 49 49 50 51 51 51 51 52 52 52 52 53 53 53 53 54 54 54 54 54 54 54 54 55 55 55 55 55 55 55